# Triage at home
A notebook demonstrating the data pipeline, using NLP capabilities such as NLTK in the domain of primary health care.
TODO: Finish of with some machine learning.

To install the requirements:
* pip install beautifulsoup4
* pip install requests
* pip install datascience
* pip install nltk

## 1. Connect to database
Fetches appropriate _code_ and _shorttitle_ from dbase and puts in a datascience.table

In [84]:
# coding: utf-8
import codecs
from datascience import * # documentation at github.com/data-8/datascience and inferentialthinking.com

import sqlite3
from sqlite3 import Error


def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return None

def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    cur.execute("SELECT code,consider,criteria,exclusion,inclusion,note,shorttitle,shorttitle_se FROM icpc")
    # cur.execute("SELECT code FROM icpc WHERE consider=?", (priority,))
 
    rows = cur.fetchall()
 
    for row in rows:
        # print(row)
        shorttitle_se.append(row[7])
        code.append(row[0])
        #print(row[0])

database = "resources/mldb.db"
code = []
shorttitle_se = []

# create a database connection
conn = create_connection(database)
with conn:
    select_all_tasks(conn)
conn.close()

icpc_table = Table().with_columns('Code', code, 'Title (Swedish)', shorttitle_se)
icpc_table

Code,Title (Swedish)
-30,Medicinsk undersökning / grundlig
-31,Medicinsk undersökning / partiell
-32,Allergitest
-33,Mikrobiologiskt /annat immunologiskt test
-34,Blodprov
-35,Urinprov
-36,Avföringsprov
-37,Histologiskt / cytologiskt prov
-38,Annat laboratorieprov UNS
-39,Kliniskt funktionstest


## 2. Processing the strings into words
Removing stopwords and stemming words to their base form.

In [85]:
import nltk
from nltk.stem.snowball import SwedishStemmer
import re

def remove_stopwords(s):
    stopwords = set(w.rstrip() for w in open('resources/stopwords_se.txt'))

    s = s.lower() # downcase
    tokens = nltk.tokenize.word_tokenize(s) # split string into words (tokens)
    tokens = [t for t in tokens if len(t) > 2] # remove short words, they're probably not useful
    # tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens] # put words into base form - don't work in Swedish though
    tokens = [t for t in tokens if t not in stopwords] # remove stopwords
    return tokens

swedish_stemmer = SwedishStemmer()	#(ignore_stopwords=False)

def stem_and_remove_stopwords(input):
    """Stems the words, removes stopwords and returns array of keywords"""
    keywords = []
    
    for item in input:
        item = item.replace(',', " ").replace('/', " ").replace('-', ' ')
        item = re.sub(' +',' ',item)

        item_keywords = ''

        for word in item.split(' '):
            stemmed = swedish_stemmer.stem(word)

            if len(remove_stopwords(stemmed)) > 0:
                item_keywords = '{0}, {1}'.format(item_keywords, remove_stopwords(stemmed)[0])

        keywords.append(item_keywords[2:])

    return keywords

stemmed_keywords = stem_and_remove_stopwords(icpc_table.column(1))
    
icpc_table = icpc_table.with_column('Keywords (stemmed)', stemmed_keywords)
icpc_table

Code,Title (Swedish),Keywords (stemmed)
-30,Medicinsk undersökning / grundlig,"medicinsk, undersökning, grund"
-31,Medicinsk undersökning / partiell,"medicinsk, undersökning, partiell"
-32,Allergitest,allergitest
-33,Mikrobiologiskt /annat immunologiskt test,"mikrobiologisk, ann, immunologisk, test"
-34,Blodprov,blodprov
-35,Urinprov,urinprov
-36,Avföringsprov,avföringsprov
-37,Histologiskt / cytologiskt prov,"histologisk, cytologisk, prov"
-38,Annat laboratorieprov UNS,"ann, laboratorieprov"
-39,Kliniskt funktionstest,"klinisk, funktionstest"


## 3. Medical history as input
Matching the stemmed words from ICPC's titles with stemmed words from the user's medical history (anamnes in Swedish)

In [86]:
#s = "Haft hosta under den senaste tiden. Är rökare och har känt sig snuvig i faktiskt flera månader och har även misstanke om någon form av allergi."
s = "Har svår smärta i buken och svettas mycket. Rädd för cancer i tarmarna."
s = remove_stopwords(s)

tags = nltk.pos_tag(s)
# print(tags)
print(nltk.ne_chunk(tags))

(S
  svår/NN
  smärta/NN
  buken/VBN
  svettas/JJ
  rädd/NN
  cancer/NN
  tarmarna/NN)


## 4. Match user input with stemmed keywords from ICPC vocabulary

In [87]:
def flatten_list(my_input):
    return [item for sublist in my_input for item in sublist]

def match_words_to_icpc(word_list, print_just_codes=False, silent=False):
    all_matches = []
    
    for word in word_list:
        stemmed_word = remove_stopwords(swedish_stemmer.stem(word))[0]
        matches = icpc_table.where('Keywords (stemmed)', are.containing(stemmed_word)).column(0)
        all_matches.append(matches)

        if len(matches) > 0 and silent is False: # only presenting words with matches in ICPC
            if print_just_codes is False:
                print("The stemmed word '{0}' matched the following codes from ICPC '{1}'".format(stemmed_word, ', '.join(matches)))
            else:
                print(matches)
                
    return all_matches
    
patient_history_in_codes = flatten_list(match_words_to_icpc(s))
#patient_history_in_codes

The stemmed word 'svår' matched the following codes from ICPC 'D21, T82'
The stemmed word 'smärt' matched the following codes from ICPC 'A01, A11, B02, D01, D02, D04, D06, F01, K01, K03, L18, N03, R01, R82, S01, U01, X01, X02, X03, X04, X18, Y01, Y02'
The stemmed word 'buk' matched the following codes from ICPC 'D01, D06, D24, D76, D91'
The stemmed word 'svett' matched the following codes from ICPC 'A09, S92'
The stemmed word 'canc' matched the following codes from ICPC 'A26, A79, B26, D26, L26, N26, R26, S26, T26, U26, X25, X26, Y26'
The stemmed word 'tarm' matched the following codes from ICPC 'D04, D16, D75, D88'


## 5. Matching result against national guidelines
For instance, 1177.se and Socialstyrelsen might be suitable. [Apply for an API key >](https://www.1177.se/Vastra-Gotaland/Om-1177/API/)

_Mind you, this cell is pretty slow even on a decent workstation_. When used in production you should make this a separate concern, perhaps another process and store the result in a dbase.

In [88]:
your_1177_api_key = "APPLY-FOR-A-CODE->" # apply for an API key at https://www.1177.se/Vastra-Gotaland/Om-1177/API/

import requests
import sys
from bs4 import BeautifulSoup
url = 'https://www.1177.se/api/artiklar/?key={0}'.format(your_1177_api_key)
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")
entries = soup.find_all('entry')

def html_decode(s):
    """
    Returns the ASCII decoded version of the given HTML string. This does
    NOT remove normal HTML tags like <p>.
    """
    htmlCodes = (
            ("'", '&#39;'),
            ('"', '&quot;'),
            ('>', '&gt;'),
            ('<', '&lt;'),
            ('&', '&amp;')
        )
    for code in htmlCodes:
        s = s.replace(code[1], code[0])
    return s

titles = []
urls = []
summary = []
contents = []
stemmed_sum = [] 
stemmed_contents = [] 

i = 0

# iterating through all items in feed
for item in entries:
    titles.append(item.title.contents[0].strip())
    try:
        summary_text = re.sub('<[^<]+?>', '', html_decode(item.summary.contents[0].strip()))
        summary.append(summary_text)
        tmp_stemmed_sum = stem_and_remove_stopwords(summary_text.split(' '))
        stemmed_sum.append(list(filter(None, tmp_stemmed_sum)))
    except:
        summary.append('')
        stemmed_sum.append('')
    
    urls.append(item.link['href'])
    text_from_html = re.sub('<[^<]+?>', '', html_decode(item.content.contents[0]))
    contents.append(text_from_html)
    tmp_stemmed_contents = stem_and_remove_stopwords(text_from_html.split(' '))
    stemmed_contents.append(list(filter(None, tmp_stemmed_contents))) if tmp_stemmed_contents not in stemmed_contents else None
    
    i += 1

tbl_1177 = Table().with_columns('Title', titles, 
                                'Summary', summary, 
                                'Contents', contents, 
                                'Stemmed summary', stemmed_sum, 
                                'Stemmed contents', stemmed_contents, 
                                'URL', urls,)
tbl_1177

Title,Summary,Contents,Stemmed summary,Stemmed contents,URL
Checklista inför utlandsresan,Här får du en översikt över vad du kan behöva tänka på ...,Sammanfattning Här får du en översikt över vad du kan ...,"['får', 'översik', 'behöv', 'tänk', 'inför', 'utlandsres ...","['sammanfattning', 'får', 'översik', 'behöv', 'tänk', 'i ...",https://www.1177.se/Tema/Vaccinationer/Allmanna-reserad/ ...
Jordnöts- och nötallergi,Reaktionen kommer ofta inom några minuter om du inte tå ...,Sammanfattning Reaktionen kommer ofta inom några minut ...,"['reaktion', 'komm', 'oft', 'någr', 'minut', 'int', 'tål ...","['sammanfattning', 'reaktion', 'komm', 'oft', 'någr', 'm ...",https://www.1177.se/Fakta-och-rad/Sjukdomar/Jordnots--oc ...
Hemorrojder,"Hemorrojder är ett vanligt, men oftast ofarligt besvär. ...","Sammanfattning Hemorrojder är ett vanligt, men oftast ...","['hemorrojd', 'van', 'oft', 'ofar', 'besvär', 'blöd', 'k ...","['sammanfattning', 'hemorrojd', 'van', 'oft', 'ofar', 'b ...",https://www.1177.se/Fakta-och-rad/Sjukdomar/Hemorrojder/
Smärta i ändtarmsöppningen,Smärta i ändtarmsöppningen är ganska vanligt och kan ha ...,Sammanfattning Smärta i ändtarmsöppningen är ganska va ...,"['smärt', 'ändtarmsöppning', 'gansk', 'van', 'fler', 'or ...","['sammanfattning', 'smärt', 'ändtarmsöppning', 'gansk', ...",https://www.1177.se/Fakta-och-rad/Sjukdomar/Smarta-i-and ...
Paracetamol,,Läkemedel som innehåller det verksamma ämnet paraceta ...,,"['läkemedel', 'innehåll', 'verksamm', 'ämnet', 'paraceta ...",https://www.1177.se/Fakta-och-rad/Lakemedel-A-O/Paracetamol/
Könsstympning – omskärelse av flickors underliv,Att omskära flickor och kvinnors underliv är en traditi ...,Sammanfattning Att omskära flickor och kvinnors underl ...,"['omskär', 'flick', 'kvinnor', 'underliv', 'tradition', ...","['sammanfattning', 'omskär', 'flick', 'kvinnor', 'underl ...",https://www.1177.se/Fakta-och-rad/Sjukdomar/Konsstympnin ...
Tandställning - tandreglering,Du kan behöva en tandställning när tänderna vuxit snett ...,Sammanfattning Du kan behöva en tandställning när tänd ...,"['behöv', 'tandställning', 'tänd', 'vuxit', 'snett', 'si ...","['sammanfattning', 'behöv', 'tandställning', 'tänd', 'vu ...",https://www.1177.se/Fakta-och-rad/Behandlingar/Tandregle ...
Graviditetstest,Vid en graviditet bildas hormonet hCG i kroppen. Genom ...,Sammanfattning Vid en graviditet bildas hormonet hCG i ...,"['graviditet', 'bild', 'hormonet', 'hcg', 'kroppen', 'ge ...","['sammanfattning', 'graviditet', 'bild', 'hormonet', 'hc ...",https://www.1177.se/Fakta-och-rad/Undersokningar/Gravidi ...
Förlossningsbrev - förlossningsplan,Under förlossningen kan du alltid prata med barnmorskan ...,Sammanfattning Under förlossningen kan du alltid prata ...,"['und', 'förlossning', 'alltid', 'prat', 'barnmorskan', ...","['sammanfattning', 'und', 'förlossning', 'alltid', 'prat ...",https://www.1177.se/Tema/Gravid/Forlossning/Praktiska-ra ...
Bipolär sjukdom,Bipolär sjukdom innebär att du är manisk och deprimerad ...,Sammanfattning Bipolär sjukdom innebär att du är manis ...,"['bipolär', 'sjukdom', 'innebär', 'manisk', 'deprimer', ...","['sammanfattning', 'bipolär', 'sjukdom', 'innebär', 'man ...",https://www.1177.se/Fakta-och-rad/Sjukdomar/Manodepressi ...


### Comparing findings from 1177.se with the patient history
We only got 100 articles to check for matches. Obviously we need to apply LSA, Word2Vec, etc on a production ready solution.

In [89]:
icpc_codes = []
for item in tbl_1177.column(3):
    try:
        codes = match_words_to_icpc(item, False, True)
        icpc_codes.append(flatten_list(codes))
        #print(codes)
    except:
        icpc_codes.append('[]')
        pass

tbl_1177 = tbl_1177.with_columns('ICPC', icpc_codes)
url_code_tbl = tbl_1177.select('URL', 'ICPC')

print('Patient history in ICPC: {}'.format(patient_history_in_codes))
matches = dict()

for match in patient_history_in_codes:
    matching_url = url_code_tbl.where('ICPC', are.containing(match))[0]
    matches[match] = matching_url
    print('For code \'{0}\' the following URL:s were found: {1}'.format(match, matching_url))

Patient history in ICPC: ['D21', 'T82', 'A01', 'A11', 'B02', 'D01', 'D02', 'D04', 'D06', 'F01', 'K01', 'K03', 'L18', 'N03', 'R01', 'R82', 'S01', 'U01', 'X01', 'X02', 'X03', 'X04', 'X18', 'Y01', 'Y02', 'D01', 'D06', 'D24', 'D76', 'D91', 'A09', 'S92', 'A26', 'A79', 'B26', 'D26', 'L26', 'N26', 'R26', 'S26', 'T26', 'U26', 'X25', 'X26', 'Y26', 'D04', 'D16', 'D75', 'D88']
For code 'D21' the following URL:s were found: ['https://www.1177.se/Fakta-och-rad/Lakemedel-A-O/Morfin/'
 'https://www.1177.se/Fakta-och-rad/Lakemedel-A-O/Oxikodon/'
 'https://www.1177.se/Fakta-och-rad/Sjukdomar/Godartad-prostataforstoring/'
 'https://www.1177.se/Fakta-och-rad/Behandlingar/Allergen-immunterapi/'
 'https://www.1177.se/Fakta-och-rad/Sjukdomar/Pollenallergi/'
 'https://www.1177.se/Fakta-och-rad/Sjukdomar/Blassten/'
 'https://www.1177.se/Fakta-och-rad/Rad-om-lakemedel/Lakemedel-vid-godartad-prostataforstoring/'
 'https://www.1177.se/Fakta-och-rad/Sjukdomar/Kallbrand/'
 'https://www.1177.se/Fakta-och-rad/Sjukdo